## The Computational Complexity of a Traveling Baseball Fan
#### Traveling Salesman Problem (TSP) 

In this notebook, Gurobi Optimizer is used to illustrate the computational complexity of the TSP. We divide the TSP into two models of (1) 30 nodes and (2) 253 nodes. to define the nodes, we use GPS coordinates of the 30 active MLB ballparks and the 253 locations around the globe which have ever held a MLB game. 

While the program uses brute force calculation on both models, an optimal solution is attainable for the 30-node model. due to our time limits, the algorithm is forced to converge on an approximate solution for the 253-node model.

In [1]:
#Imports
import pandas as pd
from collections import Counter
import geopy.distance
import itertools
import gurobipy as g

In [2]:
#Load Data
#https://www.seamheads.com/ballparks/index.php
parks = pd.read_csv("C:/Users/JTOTTEN/Desktop/MSPA/ballparks_data/Parks.csv")
teams = pd.read_csv("C:/Users/JTOTTEN/Desktop/MSPA/ballparks_data/Teams.csv")
park_teams = pd.read_csv("C:/Users/JTOTTEN/Desktop/MSPA/ballparks_data/Home_Main_Data_With_Parks_Breakout.csv")
#C:\Users\JTOTTEN\Desktop\MSPA\ballparks_data
#Get info from csvs
teamID = []
teamName = []
seasonsPlayed = []

In [3]:
for i in range(len(parks)):
    parkID = parks.iloc[i]["PARKID"]
    parksForTeam = park_teams[park_teams.Park_ID == parkID].TeamID
    teamID.append(Counter(parksForTeam).most_common(1)[0][0]) #Get team that played at home in the stadium the most
    teamName.append(teams[teams.TeamID == teamID[i]].City.values[0] + " " + teams[teams.TeamID == teamID[i]].Nickname.values[0])
    seasonsPlayed.append(sum(Counter(parksForTeam).values()))


#### Create a dataframe that lists each team, their ballpark, and their park's GPS coordinates

In [5]:
#Combine into one dataframe
df = parks.copy(deep=True)
df['TeamID'] = teamID
df['TeamName'] = teamName
df['SeasonsPlayed'] = seasonsPlayed

df.START = pd.to_datetime(parks.START, format='%Y%m%d', errors='ignore').dt.date
df.END = pd.to_datetime(parks.END, format='%Y%m%d', errors='ignore').dt.date
df['Active'] = df.END.isnull() & df.START.notnull()

In [6]:
df_active = df[df.Active == True]
for i in range(len(df_active)):
    teamName = df_active.iloc[i].TeamName
    parkName = df_active.iloc[i].NAME
    lat = df_active.iloc[i].Latitude
    long = df_active.iloc[i].Longitude
    print("The " + teamName + " play in " + parkName + " located at (" + str(lat) + ", " + str(long) + ")")


The California Angels play in Angel Stadium of Anaheim located at (33.800290000000004, -117.88268500000001)
The Texas Rangers play in Globe Life Park located at (32.751164, -97.082546)
The Atlanta Braves play in SunTrust Park located at (33.89127, -84.4681)
The Baltimore Orioles play in Oriole Park at Camden Yards located at (39.283944, -76.621572)
The Boston Red Sox play in Fenway Park located at (42.346561, -71.097337)
The Chicago Cubs play in Wrigley Field located at (41.948314, -87.655397)
The Chicago White Sox play in U.S. Cellular Field located at (41.829892, -87.633703)
The Cincinnati Reds play in Great American Ballpark located at (39.097213, -84.506483)
The Cleveland Indians play in Progressive Field located at (41.496005, -81.685326)
The Colorado Rockies play in Coors Field located at (39.756175, -104.99413)
The Detroit Tigers play in Comerica Park located at (42.339063, -83.048627)
The Houston Astros play in Minute Maid Park located at (29.757040999999997, -95.355429)
The Ka

In [7]:
coords_1 = (38.872987, -77.007435)
coords_2 = (43.641256, -79.389054)

print(geopy.distance.geodesic(coords_1, coords_2).mi) #351.61 miles

351.606681455


#### Setup Gurobi base functions

In [8]:
# Copyright 2018, Gurobi Optimization, LLC

# Solve a traveling salesman problem on a randomly generated set of
# points using lazy constraints.   The base MIP model only includes
# 'degree-2' constraints, requiring each node to have exactly
# two incident edges.  Solutions to this model may contain subtours -
# tours that don't visit every city.  The lazy constraint callback
# adds new constraints to cut them off.

# Callback - use lazy constraints to eliminate sub-tours

def subtourelim(model, where):
    if where == g.GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = g.tuplelist((i,j) for i,j in model._vars.keys() if vals[i,j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n:
            # add subtour elimination constraint for every pair of cities in tour
            model.cbLazy(g.quicksum(model._vars[i,j]
                                  for i,j in itertools.combinations(tour, 2))
                         <= len(tour)-1)


# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = list(range(n))
    cycle = range(n+1) # initial length has 1 more city
    while unvisited: # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i,j in edges.select(current,'*') if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

#### Solve TSP graph with 30-vertices (all current baseball parks)

In [9]:
#All Current
    
n = len(df_active)
points = [(df_active.iloc[i].Latitude, df_active.iloc[i].Longitude) for i in range(n)]
    
dist = {(i,j) :
    geopy.distance.geodesic((points[i][0],points[i][1]), (points[j][0],points[j][1])).mi
    for i in range(n) for j in range(i)}

   
m = g.Model()

# Create variables
vars = m.addVars(dist.keys(), obj=dist, vtype=g.GRB.BINARY, name='e')
for i,j in vars.keys():
    vars[j,i] = vars[i,j] # edge in opposite direction

# Add degree-2 constraint
m.addConstrs(vars.sum(i,'*') == 2 for i in range(n))
m.update()

In [10]:
# Optimize model
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

vals = m.getAttr('x', vars)
selected = g.tuplelist((i,j) for i,j in vals.keys() if vals[i,j] > 0.5)

tour = subtour(selected)
assert len(tour) == n

print('')
print('Optimal tour: %s' % str(tour))
print('Optimal cost: %g' % m.objVal)
print('')

for cityIndx in tour:
    print(df_active.iloc[cityIndx].NAME + ", " + df_active.iloc[cityIndx].CITY)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 30 rows, 435 columns and 870 nonzeros
Variable types: 0 continuous, 435 integer (435 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 30 rows, 435 columns, 870 nonzeros
Variable types: 0 continuous, 435 integer (435 binary)

Root relaxation: objective 7.939909e+03, 42 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7939.90870    0    8          - 7939.90870      -     -    0s
     0     0 8158.54393    0    6          - 8158.54393      -     -    0s
     0     0 8692.72658    0    6          - 8692.72658      -     -    0s
     0     0 8822.17939    0    8          - 8822.17939      -     - 

#### Approximate optimal path for TSP graph with 253 vertices
The constraint 'TimeLimit' tells the algorithm to converge on an approximation if the limit is reached. In this example, 5 minutes is used.

In [11]:
#All Historical
    
n = len(df)
points = [(df.iloc[i].Latitude, df.iloc[i].Longitude) for i in range(n)]
    
dist = {(i,j) :
    geopy.distance.geodesic((points[i][0],points[i][1]), (points[j][0],points[j][1])).mi
    for i in range(n) for j in range(i)}

m = g.Model()
m.setParam('TimeLimit', 5*60) #set time limit

# Create variables
vars = m.addVars(dist.keys(), obj=dist, vtype=g.GRB.BINARY, name='e')
for i,j in vars.keys():
    vars[j,i] = vars[i,j] # edge in opposite direction

# Add degree-2 constraint
m.addConstrs(vars.sum(i,'*') == 2 for i in range(n))
m.update()

# Optimize model
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

vals = m.getAttr('x', vars)
selected = g.tuplelist((i,j) for i,j in vals.keys() if vals[i,j] > 0.5)

tour = subtour(selected)
assert len(tour) == n

print('')
print('Optimal tour: %s' % str(tour))
print('Optimal cost: %g' % m.objVal)
print('')

for cityIndx in tour:
    print(df.iloc[cityIndx].NAME + ", " + df.iloc[cityIndx].CITY)

Changed value of parameter TimeLimit to 300.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 253 rows, 31878 columns and 63756 nonzeros
Variable types: 0 continuous, 31878 integer (31878 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-03, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.07s
Presolved: 253 rows, 31878 columns, 63756 nonzeros
Variable types: 0 continuous, 31878 integer (31878 binary)

Root relaxation: objective 2.161334e+04, 316 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 21613.3446    0   32          - 21613.3446      -     -    0s
     0     0 22200.4832    0   24          - 22200.4832      -     -    1s
     0     0 22201.3488

Wright Street Grounds, Milwaukee
Miller Park, Milwaukee
County Stadium, Milwaukee
Athletic Park, Minneapolis
Ft. Street Grounds, St. Paul
Metropolitan Stadium, Bloomington
Hubert H. Humphrey Metrodome, Minneapolis
Target Field, Minneapolis
Kingdome, Seattle
Safeco Field, Seattle
Sicks' Stadium, Seattle
Tokyo Dome, Tokyo
Sydney Cricket Ground, Sydney
Aloha Stadium, Honolulu
Candlestick Park, San Francisco
Seals Stadium, San Francisco
AT&T Park, San Francisco
Oakland-Alameda County Coliseum, Oakland
Dodger Stadium, Los Angeles
Los Angeles Memorial Coliseum, Los Angeles
Wrigley Field, Los Angeles
Angel Stadium of Anaheim, Anaheim
Petco Park, San Diego
Jack Murphy Stadium, San Diego
Cashman Field, Las Vegas
Chase Field, Phoenix
Estadio Monterrey, Monterrey
Colt Stadium, Houston
Minute Maid Park, Houston
Astrodome, Houston
Arlington Stadium, Arlington
Globe Life Park, Arlington
Mile High Stadium, Denver
Coors Field, Denver
Athletic Park, Kansas City
Association Park, Kansas City
Exposition 

## Discussions and Conclusions
The purpose of this project was to explore TSP and the computational complexity surrounding it. The
exercise analyzed two sizes of a TSP, (1) a 30-ballpark problem and (2) a 253-ballpark problem. As
mentioned, the program converged on a solution in less than a second for the 30-ballpark problem.
However, the larger problem failed to converge in as long as 12 hours. As a result, candidate solutions
for the larger problem were obtained through solver runs of varying times; candidate solutions saw their
errors decrease in proportion with the solver time that had been allotted.

TSP optimization is an NP-Hard problem; regardless of the size of the problem. As it
stands, solutions cannot be presented in polynomial time nor can solutions be verified in polynomial
time. The factorial nature of the TSP creates computational complexity which limits modern computing
like our program. Our problem was relatively simplistic in its setup. It was considered
symmetric; it did not investigate transitions between various modes of travel; it did not reconcile the
feasibility of applying Manhattan distance measures in urban areas. Despite the problem was stripped to
the simplest TSP interpretation, the larger problem could not converge to an optimal solution.